In [ ]:
import tensorflow as tf 
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt 
import skimage.transform as skTrans

In [ ]:
trainedModelPath = ".\\trained_model\\"
rootDirPath = "..\\mmwhs\\ct_train\\"

In [ ]:
unetAxial = tf.keras.models.load_model(os.path.join(trainedModelPath, "unetAxial.h5"))
unetAxial.summary()

In [ ]:
def ConvertPredictToImage(sliceImagePredict):
    def MapArgmax(x):
        encodeList = {
            0: 0,
            1: 500,
            2: 600,
            3: 420,
            4: 550,
            5: 205,
            6: 820,
            7: 850,
        }
        return encodeList.get(x, None)

    a = sliceImagePredict.copy()
    a = a.argmax(axis=-1)
    a = np.vectorize(MapArgmax)(a)
    return a

In [ ]:
x = np.load("./x_axial.npy")
y = np.load("./y_axial.npy")

In [ ]:
sliceImage = np.squeeze(x[300], axis=-1)
sliceLabel = ConvertPredictToImage(y[300])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 6))
axes[0].set_title("Hình chụp ban đầu")
axes[0].imshow(sliceImage, cmap="gray")
axes[1].set_title("Các phân đoạn đã được kiểm định")
axes[1].imshow(sliceLabel, cmap="gray")

In [ ]:
sliceImageForPredict = [sliceImage]
sliceImageForPredict = np.expand_dims(sliceImageForPredict, -1)
print(sliceImageForPredict.shape)

In [ ]:
sliceImagePredict = unetAxial.predict(sliceImageForPredict)
sliceImagePredict = sliceImagePredict[0]

In [ ]:
sliceImagePredict = ConvertPredictToImage(sliceImagePredict)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 6))
axes[0].set_title("Hình chụp ban đầu")
axes[0].imshow(sliceImage, cmap="gray")
axes[1].set_title("Các phân đoạn đã được kiểm định")
axes[1].imshow(sliceLabel, cmap="gray")
axes[2].set_title("Các phân đoạn từ mô hình học sâu")
axes[2].imshow(sliceImagePredict, cmap="gray")

In [ ]:
print(f"Unique label: {np.unique(sliceLabel)}, Unique predict value: {np.unique(sliceImagePredict)}")

In [ ]:
def DiceCoefficient(y_true, y_pred, class_id):
    # Create binary masks for the specific class
    y_true_class = np.where(y_true == class_id, 1, 0)
    y_pred_class = np.where(y_pred == class_id, 1, 0)

    # Calculate the intersection between the masks
    intersection = np.logical_and(y_true_class, y_pred_class).sum()

    # Calculate the sum of elements in each mask
    sum_y_true = np.sum(y_true_class)
    sum_y_pred = np.sum(y_pred_class)

    # Calculate the Dice coefficient for the class
    # Adding a small epsilon to avoid division by zero
    dice = (2.0 * intersection) / (sum_y_true + sum_y_pred + 1e-8)

    return dice


def DiceMultiClass(y_true, y_pred, num_classes):
    dice_scores = []
    for class_id in range(num_classes):
        dice = DiceCoefficient(y_true, y_pred, class_id)
        dice_scores.append(dice)

    # Calculate the average Dice coefficient across all classes
    average_dice = np.mean(dice_scores)
    return average_dice